In [ ]:
only one heavy chain antibody

In [36]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [49]:
%load_ext autoreload
%autoreload 2 

import os
import sys

curr_dir = os.getcwd()
print('current direcotry is ', curr_dir)
if curr_dir not in sys.path:
    sys.path.append(curr_dir)
src_dir = os.path.join(curr_dir, 'src')
if src_dir not in sys.path:
    sys.path.append(src_dir)
print(sys.path)

#
pdb_dir = './pdb'
data_dir = './data'
outputs_dir = './outputs'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
current direcotry is  /home/yuan/bio/protein_design/docking_antigen-antibody/antibody_antigen_complex
['/home/yuan/anaconda3/envs/openfold-env/lib/python310.zip', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/lib-dynload', '', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages/openfold-2.2.0-py3.10-linux-x86_64.egg', '/home/yuan/bio/protein_design/docking_antigen-antibody/antibody_antigen_complex', '/home/yuan/bio/protein_design/docking_antigen-antibody/antibody_antigen_complex/src']


## view structure

In [50]:
pdb_acc = '1RI8'

In [4]:
# get *pdb
pdb_file = f'pdb/{pdb_acc}.pdb'
if not os.path.isfile(pdb_file):
    print(f"download {pdb_acc} to {pdb_file}")
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_acc}.pdb -P ./pdb")
print(f"{pdb_file} is ready")

download 1RI8 to pdb/1RI8.pdb
pdb/1RI8.pdb is ready


In [5]:
import py3Dmol
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open(f'pdb/{pdb_acc}.pdb').read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## parse complex


In [51]:
# get chains
from src.parse_abag import ParseAbAg

p = ParseAbAg(f'./pdb/{pdb_acc}.pdb', outputs_dir)

# get chains
chains = p.get_chains()
pprint(chains)

Successfully retrieve structure of 1RI8
outputs dir:  ./outputs/1R/1RI8
[{'chains': [{'AA residues': 234,
              'chain_id': 'A',
              'seq': 'VQLVESGGGSVQAGGSLRLSCAVSGYKDRNYCMGWFRRAPGKEREGVAVIDSSGRTAYADSVKGRFTISRDVALDTAYLQMNSLKPEDTAMYYCAAGWSSLGSCGTNRNRYNYWGQGTQVTVSS'},
             {'AA residues': 251,
              'chain_id': 'B',
              'seq': 'KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCKGTDVQAWIRGCRL'}],
  'model_id': 0}]


In [5]:
print(p.structure.header['keywords'])
print(p.structure_id)

beta sandwich, immunoglobulin fold, vhh-lysozyme complex, alpha-beta orthogonal bundle, immune system-hydrolase complex
1RI8


In [28]:
p.structure.header['compound']

{'1': {'misc': '',
  'molecule': 'camelid antibody heavy chain',
  'chain': 'a',
  'fragment': 'vhh domain, variable domain',
  'synonym': '1d2l19',
  'engineered': 'yes'},
 '2': {'misc': '',
  'molecule': 'lysozyme c',
  'chain': 'b',
  'synonym': '1,4-beta-n-acetylmuramidase c, allergen gal d 4, gal d iv',
  'ec_number': '3.2.1.17',
  'ec': '3.2.1.17'}}

In [52]:
# filter: is antibody-antigen complex
info = p.filter_antibody_antigen()
print(info)
p.update_chains('chain_type')

[{'A': 'heavy', 'B': 'receptor'}]


In [53]:
# split chains
p.chains_to_pdb()

Saved chain ['A'] to ./outputs/1R/1RI8/1RI8_A.pdb
Saved chain ['B'] to ./outputs/1R/1RI8/1RI8_B.pdb
Saved chain ['A', 'B'] to ./outputs/1R/1RI8/1RI8_A_B.pdb


In [42]:
p.bounded

[{'type': 'heavy-receptor',
  'chains': [['A'], ['B'], ['A', 'B']],
  'pdb_files': ['./outputs/1R/1RI8/1RI8_A.pdb',
   './outputs/1R/1RI8/1RI8_B.pdb',
   './outputs/1R/1RI8/1RI8_A_B.pdb']}]

## load SASA
heavy chain ~ receptor
A ~ B

In [54]:
# calculate SASA
p.build_freesasa()

'./outputs/1R/1RI8/freesasa.p'

In [55]:
from analyze_sasa import AnalyzeSasa

p2= AnalyzeSasa('./outputs/1R/1RI8/freesasa.p')
p2.print_meta()

number of complex 1
heavy chain A ~ receptor chain B


In [46]:
# complex 0
p2.print_delta_sasa(0)

complex: heavy chain A ~ receptor chain B
------------------------------
heavy chain:
    chain_id res_no  res aa      value
42         A     44  GLU  E   4.055821
48         A     50  VAL  V   1.351940
50         A     52  ASP  D   6.300728
54         A     56  ARG  R  40.691982
55         A     57  THR  T  12.483747
56         A     58  ALA  A  32.446570
57         A     59  TYR  Y  19.311244
58         A     60  ALA  A   1.351940
59         A     61  ASP  D  36.844473
97         A     99  TRP  W  72.956113
98         A    100  SER  S   0.999328
99         A    101  SER  S  22.501946
102        A    104  SER  S  38.892314
103        A    105  CYS  C  16.856479
104        A    106  GLY  G  20.279106
105        A    107  THR  T  42.564107
106        A    108  ASN  N  60.746765
107        A    109  ARG  R  31.356010
108        A    110  ASN  N  83.273895
109        A    111  ARG  R  81.789291
110        A    112  TYR  Y   6.995296
111        A    113  ASN  N  10.815523
-----------------

In [47]:
# retrieve sequences
motifs = p2.retrieve_seq()
motifs

([{'heavy': [{'seq': 'EREGVAVIDSSGRTAYAD',
     'sig_res': 9,
     'start': (42, '44'),
     'end': (59, '61')},
    {'seq': 'WSSLGSCGTNRNRYN',
     'sig_res': 13,
     'start': (97, '99'),
     'end': (111, '113')}],
   'receptor': [{'seq': 'KFESNFNTQATNRNTDGSTDYGILQ',
     'sig_res': 11,
     'start': (32, '33'),
     'end': (56, '57')},
    {'seq': 'VAWRNR',
     'sig_res': 4,
     'start': (108, '109'),
     'end': (113, '114')}]}],
 './outputs/1R/1RI8/seq_sasa.p')

## distance

In [56]:
# calculate distance
dist_file = p.build_dist()
print(dist_file)

./outputs/1R/1RI8/dist.p


In [57]:
from analyze_dist import AnalyzeDist

max_span = 30
p3= AnalyzeDist(dist_file, max_span=max_span)
p3.print_meta()

number of complex 1
heavy chain A ~ receptor chain B


In [58]:
# complex
p3.print_min_dist(0)

A-B
------------------------------
     res_no  res atom      value aa
0         2  VAL    C  12.188063  V
1         3  GLN    C  14.470847  Q
2         4  LEU    C  11.902370  L
3         5  VAL    C  17.289200  V
4         6  GLU    C  15.468970  E
..      ...  ...  ...        ... ..
119     121  VAL    C  19.711534  V
120     122  THR    C  22.665323  T
121     123  VAL    C  21.797189  V
122     124  SER    C  26.069666  S
123     125  SER    C  26.977037  S

[124 rows x 5 columns]
------------------------------
     res_no  res atom     value aa
32       33  LYS    C  3.883477  K
33       34  PHE    C  2.940978  F
34       35  GLU    C  3.330475  E
36       37  ASN    C  3.170125  N
42       43  THR    C  4.555289  T
43       44  ASN    C  2.717124  N
44       45  ARG    C  3.103508  R
45       46  ASN    C  3.191467  N
46       47  THR    C  2.691477  T
47       48  ASP    C  3.488377  D
51       52  ASP    C  4.196784  D
108     109  VAL    C  3.460824  V
109     110  ALA    C  

In [59]:
# retrieve sequences
motifs = p3.retrieve_seq()
motifs[0]

Sequence-Distance data: ./outputs/1R/1RI8/seq_dist.p.


[{'A': [{'seq': 'D',
    'sig_res': 1,
    'start': (50, 52),
    'end': (50, 52),
    'cutoff': 3},
   {'seq': 'CGTNRN',
    'sig_res': 4,
    'start': (103, 105),
    'end': (108, 110),
    'cutoff': 3}],
  'B': [{'seq': 'FESNFNTQATNRNT',
    'sig_res': 3,
    'start': (33, 34),
    'end': (46, 47),
    'cutoff': 3},
   {'seq': 'R',
    'sig_res': 1,
    'start': (113, 114),
    'end': (113, 114),
    'cutoff': 3}]},
 {'A': [{'seq': 'VIDSSGRTAYAD',
    'sig_res': 7,
    'start': (48, 50),
    'end': (59, 61),
    'cutoff': 4},
   {'seq': 'WSSLGSCGTNRNRYN',
    'sig_res': 12,
    'start': (97, 99),
    'end': (111, 113),
    'cutoff': 4}],
  'B': [{'seq': 'KFESNFNTQATNRNTD',
    'sig_res': 9,
    'start': (32, 33),
    'end': (47, 48),
    'cutoff': 4},
   {'seq': 'VAWRNR',
    'sig_res': 4,
    'start': (108, 109),
    'end': (113, 114),
    'cutoff': 4}]},
 {'A': [{'seq': 'VIDSSGRTAYAD',
    'sig_res': 8,
    'start': (48, 50),
    'end': (59, 61),
    'cutoff': 5},
   {'seq': 'WSSL